In [6]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri = MLFLOW_TRACKING_URI)

In [3]:
#client.create_experiment(name="my-cool-experiment")

In [7]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 6.8",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [8]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")


run id: d50372ec18eb48a7916aa1b039f8e134, rmse: 6.3048
run id: 183119b304d34106ac0bf7f0b289910f, rmse: 6.3050
run id: 8fedd5fb83e54a56a56d76876fab6557, rmse: 6.3075
run id: 2a39a55b83024386add36e24637a104b, rmse: 6.3087
run id: 9f70c762b29e48b08109375a8d58a3e9, rmse: 6.3093


In [9]:
import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [10]:
run_id = "2a39a55b83024386add36e24637a104b"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="taxi-regressor")

Registered model 'taxi-regressor' already exists. Creating a new version of this model...
Created version '2' of model 'taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1723822783974, current_stage='None', description=None, last_updated_timestamp=1723822783974, name='taxi-regressor', run_id='2a39a55b83024386add36e24637a104b', run_link=None, source='/workspaces/mlops-learn/01-intro/mlruns/1/2a39a55b83024386add36e24637a104b/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [11]:
model_uri

'runs:/2a39a55b83024386add36e24637a104b/model'

In [14]:
model_name = "taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)
for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")
    

version: 2, stage: None


/tmp/ipykernel_7418/229625682.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [15]:
from sklearn.metrics import mean_squared_error
import pandas as pd

In [27]:
def read_dataframe(filename):
    #df = pd.read_csv(filename)
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(run_id, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"runs:/{run_id}/models_mlflow")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [19]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [21]:
run_id = "d3db079a75e747bea8d61ae1a5c18d0a" # this is the run id for version 1, for what we have an artifact for
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/workspaces/mlops-learn/01-intro/preprocessor'

In [22]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [23]:
X_test = preprocess(df, dv)

In [24]:
target = "duration"
y_test = df[target].values

In [28]:
%time test_model(run_id=run_id, X_test=X_test, y_test=y_test)

CPU times: user 637 ms, sys: 224 ms, total: 861 ms
Wall time: 1.78 s


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 9.262670438537066}